## 1. Model Yükleme

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Kaydedilmiş modeli yükle
model_path = '../models/wear_gradientboosting_tuned_model.joblib'
loaded_data = joblib.load(model_path)
model = loaded_data['model']
feature_cols = loaded_data['features']

print('Model başarıyla yüklendi!')
print('Gerekli özellikler:', feature_cols)
print('Model tipi:', type(model).__name__)

## 2. Test Verisi ile Değerlendirme

In [ ]:
# Orijinal veriyi yükle ve test setini oluştur
data_path = '/Users/onurcan/Documents/GDG_HATAY/Dataset.xlsx'
df = pd.read_excel(data_path)
df_clean = df.dropna()

from sklearn.model_selection import train_test_split
X = df_clean[feature_cols]
y = df_clean['AşınmaOranı']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Test seti üzerinde tahmin yap
y_pred = model.predict(X_test)

# Metrikleri hesapla
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
r2 = r2_score(y_test, y_pred)

print('Test Seti Performansı')
print('='*50)
print(f'MAE:  {mae:.3f}')
print(f'RMSE: {rmse:.3f}')
print(f'R²:   {r2:.3f}')
print(f'Test örnekleri: {len(y_test)}')

## 3. Tahmin vs Gerçek Görselleştirme

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sol: Scatter plot
axes[0].scatter(y_test, y_pred, alpha=0.5, edgecolor='k', s=50)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Mükemmel Tahmin')
axes[0].set_xlabel('Gerçek Aşınma Oranı', fontsize=12)
axes[0].set_ylabel('Tahmin Edilen Aşınma Oranı', fontsize=12)
axes[0].set_title(f'Gerçek vs Tahmin (R²={r2:.3f})', fontsize=14)
axes[0].legend()
axes[0].grid(alpha=0.3)

# Sağ: Hata dağılımı
residuals = y_test - y_pred
axes[1].hist(residuals, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Sıfır Hata')
axes[1].set_xlabel('Hata (Gerçek - Tahmin)', fontsize=12)
axes[1].set_ylabel('Frekans', fontsize=12)
axes[1].set_title('Hata Dağılımı', fontsize=14)
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print('\nHata İstatistikleri:')
print(residuals.describe())

## 4. Yeni Veri ile Tahmin (Manuel Girdi)

In [ ]:
# Örnek yeni veriler (manuel)
new_data = pd.DataFrame({
    'Kesme Gücü': [12.5, 8.0, 15.0],
    'İlerleme Hızı': [120.0, 60.0, 140.0],
    'RPM': [1200, 1100, 1250]
})

print('Yeni Veri:')
display(new_data)

# Tahmin yap
predictions = model.predict(new_data)

# Sonuçları göster
results = new_data.copy()
results['Tahmin Edilen Aşınma Oranı'] = predictions

print('\nTahmin Sonuçları:')
display(results)

for i, pred in enumerate(predictions, 1):
    print(f'Örnek {i}: Aşınma Oranı = {pred:.3f}')

## 5. Gerçek Zamanlı Tahmin Fonksiyonu

In [ ]:
def predict_wear_rate(kesme_gucu, ilerleme_hizi, rpm):
    """
    Verilen CNC parametreleri için aşınma oranını tahmin eder.
    
    Parametreler:
    - kesme_gucu: Kesme gücü (kW)
    - ilerleme_hizi: İlerleme hızı (mm/min)
    - rpm: Devir/dakika
    
    Döndürür:
    - Tahmin edilen aşınma oranı
    """
    input_data = pd.DataFrame({
        'Kesme Gücü': [kesme_gucu],
        'İlerleme Hızı': [ilerleme_hizi],
        'RPM': [rpm]
    })
    
    prediction = model.predict(input_data)[0]
    return prediction

# Fonksiyonu test et
print('Fonksiyon Testi:')
print('='*50)
test_cases = [
    (10.0, 100.0, 1150),
    (15.0, 130.0, 1200),
    (5.0, 50.0, 1080)
]

for kg, ih, r in test_cases:
    pred = predict_wear_rate(kg, ih, r)
    print(f'Kesme Gücü={kg} kW, İlerleme Hızı={ih} mm/min, RPM={r} → Aşınma Oranı: {pred:.3f}')

## 6. Toplu Tahmin (Batch Prediction)

In [ ]:
# Test setinin tamamı üzerinde detaylı analiz
results_df = pd.DataFrame({
    'Gerçek': y_test.values,
    'Tahmin': y_pred,
    'Hata': y_test.values - y_pred,
    'Mutlak Hata': np.abs(y_test.values - y_pred)
})

# En iyi ve en kötü tahminler
print('En İyi 5 Tahmin (En Düşük Mutlak Hata):')
display(results_df.nsmallest(5, 'Mutlak Hata'))

print('\nEn Kötü 5 Tahmin (En Yüksek Mutlak Hata):')
display(results_df.nlargest(5, 'Mutlak Hata'))

# Hata aralıklarına göre dağılım
print('\nHata Aralıklarına Göre Dağılım:')
print('='*50)
bins = [0, 0.5, 1.0, 2.0, np.inf]
labels = ['Çok İyi (<0.5)', 'İyi (0.5-1.0)', 'Orta (1.0-2.0)', 'Zayıf (>2.0)']
results_df['Kategori'] = pd.cut(results_df['Mutlak Hata'], bins=bins, labels=labels)
print(results_df['Kategori'].value_counts())

## 7. Model Özeti ve Öneriler

In [ ]:
print('MODEL PERFORMANS ÖZETİ')
print('='*70)
print(f'Model Tipi: {type(model).__name__}')
print(f'Özellikler: {", ".join(feature_cols)}')
print(f'Test Örnekleri: {len(y_test)}')
print()
print('Metrikler:')
print(f'  • Ortalama Mutlak Hata (MAE): {mae:.3f}')
print(f'  • Kök Ortalama Kare Hata (RMSE): {rmse:.3f}')
print(f'  • Belirlilik Katsayısı (R²): {r2:.3f}')
print()
print('Yorum:')
if r2 > 0.7:
    print('  ✅ Model yüksek doğrulukla tahmin yapıyor.')
elif r2 > 0.5:
    print('  ⚠️  Model makul seviyede tahmin yapıyor, iyileştirme mümkün.')
else:
    print('  ❌ Model performansı düşük, yeniden eğitim veya özellik mühendisliği gerekli.')
print()
print('Öneriler:')
print('  1. Yeni veri toplandıkça modeli periyodik olarak yeniden eğitin')
print('  2. Ekstrem değerlerde (outliers) tahmin hatası artabilir')
print('  3. Üretim ortamında model versiyonlarını takip edin')
print('  4. A/B testi ile farklı model versiyonlarını karşılaştırın')